In [1]:
from datasets import load_dataset
dataset = load_dataset("shunk031/JGLUE", 'JCommonsenseQA')

Found cached dataset jglue (/home/lan/.cache/huggingface/datasets/shunk031___jglue/JCommonsenseQA/1.1.0/3bec28582f7c64bc25fb042e3fffdbcd04c45bce4e494a6c6baa8ccdc53185ca)


  0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
def build_prompt(row):
    return f"""以下の問いを読んで、A-D選択肢からあてはまるものを一つ選び、回答してください：

# 質問:
主に子ども向けのもので、イラストのついた物語が書かれているものはどれ？

A. 世界
B. 写真集
C. 絵本
D. 論文
E. 図鑑

# 回答:
C

#  質問:
未成年者を監護・教育し，彼らを監督し，彼らの財産上の利益を守る法律上の義務をもつ人は？

A. 浮浪者
B. 保護者
C. お坊さん
D. 宗教者
E. 預言者

# 回答:
B

#  質問:
数字の１を表すときに使う体は？

A. 胸
B. 肉球
C. 背中
D. 人差し指
E. 親指

# 回答:
D

#  質問:
{row['question']}

A. {row['choice0']}
B. {row['choice1']}
C. {row['choice2']}
D. {row['choice3']}
E. {row['choice4']}

# 回答:
"""

print(build_prompt(dataset['train'][0]))

以下の問いを読んで、A-D選択肢からあてはまるものを一つ選び、回答してください：

# 質問:
主に子ども向けのもので、イラストのついた物語が書かれているものはどれ？

A. 世界
B. 写真集
C. 絵本
D. 論文
E. 図鑑

# 回答:
C

#  質問:
未成年者を監護・教育し，彼らを監督し，彼らの財産上の利益を守る法律上の義務をもつ人は？

A. 浮浪者
B. 保護者
C. お坊さん
D. 宗教者
E. 預言者

# 回答:
B

#  質問:
数字の１を表すときに使う体は？

A. 胸
B. 肉球
C. 背中
D. 人差し指
E. 親指

# 回答:
D

#  質問:
主に子ども向けのもので、イラストのついた物語が書かれているものはどれ？

A. 世界
B. 写真集
C. 絵本
D. 論文
E. 図鑑

# 回答:



In [3]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import AutoTokenizer, TextGenerationPipeline
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
import logging

def load_model(name_or_path='TheBloke/Llama-2-13B-GPTQ', device='cuda:0'):
    tokenizer = AutoTokenizer.from_pretrained(name_or_path, trust_remote_code=True)
    model = AutoGPTQForCausalLM.from_quantized(name_or_path,
                                               use_safetensors=True,
                                               device=device)
    return tokenizer, model

tokenizer, model = load_model()

The safetensors archive passed at /home/lan/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-GPTQ/snapshots/36a2c04e7a318e70e4e3bbe5a97fddd5c68d4c9c/gptq_model-4bit-128g.safetensors does not contain metadata. Make sure to save your model with the `save_pretrained` method. Defaulting to 'pt' metadata.
skip module injection for FusedLlamaMLPForQuantizedModel not support integrate without triton yet.


In [4]:
def generate(prompt):
    return tokenizer.decode(
        model.generate(**tokenizer(prompt, return_tensors="pt").to(model.device),
                       temperature=0.1,
                       max_new_tokens=3)[0])

In [9]:
mappings = {'A':0,'B':1,'C':2,'D':3,'E':4}

def run_test(item, verbose=False):
    res = generate(build_prompt(item))

    if verbose:
        print(item)
        print(res)

    answer = res.split('# 回答:')[-1].strip()
    
    if answer in mappings:
        final_answer = mappings[answer]
    else:
        print(f'Cannot parse answer<{answer}> for {item}')
        final_answer = None

    if final_answer == item['label']:
        return 1
    else:
        print('wrong sample', item, answer)
        return 0

run_test(dataset['validation'][0], verbose=True)

{'q_id': 8939, 'question': '電子機器で使用される最も主要な電子回路基板の事をなんと言う？', 'choice0': '掲示板', 'choice1': 'パソコン', 'choice2': 'マザーボード', 'choice3': 'ハードディスク', 'choice4': 'まな板', 'label': 2}
<s> 以下の問いを読んで、A-D選択肢からあてはまるものを一つ選び、回答してください：

# 質問:
主に子ども向けのもので、イラストのついた物語が書かれているものはどれ？

A. 世界
B. 写真集
C. 絵本
D. 論文
E. 図鑑

# 回答:
C

#  質問:
未成年者を監護・教育し，彼らを監督し，彼らの財産上の利益を守る法律上の義務をもつ人は？

A. 浮浪者
B. 保護者
C. お坊さん
D. 宗教者
E. 預言者

# 回答:
B

#  質問:
数字の１を表すときに使う体は？

A. 胸
B. 肉球
C. 背中
D. 人差し指
E. 親指

# 回答:
D

#  質問:
電子機器で使用される最も主要な電子回路基板の事をなんと言う？

A. 掲示板
B. パソコン
C. マザーボード
D. ハードディスク
E. まな板

# 回答:
C




1

In [11]:
import tqdm

test_ds = dataset['validation']
results = []

for i in tqdm.notebook.tqdm(range(len(test_ds))):
    results.append(run_test(test_ds[i]))


  0%|          | 0/1119 [00:00<?, ?it/s]

wrong sample {'q_id': 8941, 'question': 'しゃがんだりする様を何という？', 'choice0': '腰を下す', 'choice1': '座る', 'choice2': '仮眠を取る', 'choice3': '寝る', 'choice4': '起きる', 'label': 0} D
wrong sample {'q_id': 8945, 'question': '乗り物が関係する言葉はどれ？', 'choice0': '外に出る', 'choice1': '歩く', 'choice2': '靴を脱ぐ', 'choice3': '靴を履く', 'choice4': '車から降りる', 'label': 4} D
wrong sample {'q_id': 8961, 'question': 'お金を？', 'choice0': '払い出す', 'choice1': '明かりを灯す', 'choice2': '鑑賞する', 'choice3': '買い求める', 'choice4': '償還', 'label': 0} E
wrong sample {'q_id': 8965, 'question': 'この中で鳥取にあって岡山や兵庫にないものは？', 'choice0': '関西地方', 'choice1': '砂丘', 'choice2': '後楽園', 'choice3': 'ポートタワー', 'choice4': '県', 'label': 1} E
wrong sample {'q_id': 8968, 'question': '同じ部首が使われている熟語はどれですか？', 'choice0': '学習', 'choice1': '把握', 'choice2': '理解', 'choice3': '問題', 'choice4': '啓蒙', 'label': 1} E
wrong sample {'q_id': 8970, 'question': '間違って理解されてしまう、と言えばどれ？', 'choice0': '誤解', 'choice1': '更年期', 'choice2': '死', 'choice3': '夢', 'choice4': '嘘', 'label': 0} E
wrong sample {'q

In [17]:
total = len(results)
correct = sum(results)

print(f'Correct: {correct}/{total}, Accuracy: {correct/total}')

Correct: 853/1119, Accuracy: 0.7622877569258266
